# 3D Data Processing

---
A.A. 2024 - Wanmeng Li, Daniel Fusaro
---


## Lab - PointNet: Point Cloud 3D Descriptors

original paper -
[PointNet: Deep Learning on Point Sets for 3D Classification and Segmentation](https://web.stanford.edu/~rqi/pointnet/)

dataset link: https://drive.google.com/drive/folders/1IweJGcOeOZN3wY79i2jFt3JE1bd7G51Z?usp=share_link

SHOT descriptions: [lecture](https://stem.elearning.unipd.it/pluginfile.php/517107/mod_resource/content/25/Slide_3DP_13_3D%20Local%20Descriptors.pdf)

To add a link to the dataset in your Google Drive main folder, you need to:

 - Click on the link
 - Right click on "dataset"
 - Click Add shortcut to Drive

When you will mount your drive folder in Colab you will find this folder without the need of re-uploading it.

In [ ]:
import numpy as np
import random
import math
import time
import copy
import os
import glob

import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R

# pyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# a nice training progress bar
from tqdm import tqdm, trange

In [ ]:
running_on_colab = True

if running_on_colab:
    # useful for visualization
    ## note: it's not necessary to restart the notebook environment after installation
    !pip install open3d

    import plotly.graph_objects as go

# visualization
import open3d as o3d


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.4 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


# Connect and mount your Google Drive

In [ ]:
if running_on_colab:
    from google.colab import drive
    drive_path = '/content/drive'
    drive.mount(drive_path)

Mounted at /content/drive


# General Parameters

In [ ]:
if running_on_colab:
    dataset_path_train = os.path.join(drive_path, "MyDrive", "3D_dp_dataset", "3dshapes", "train")
    dataset_path_valid = os.path.join(drive_path, "MyDrive", "3D_dp_dataset", "3dshapes", "valid")
    dataset_path_test = os.path.join(drive_path,  "MyDrive", "3D_dp_dataset", "3dshapes", "test")
else:
    dataset_path_train = os.path.join("datasets", "train")
    dataset_path_valid = os.path.join("datasets", "valid")
    dataset_path_test = os.path.join("datasets", "test")

# Visualization Example
In order to visualize colored point clouds we make use of the Python package *Open3D*.

Unfortunately, the original doesn't run on Colab.
So, we replace the drawing function with a custom one (*draw_geometries*) that allows for rendering here on Colab.



In [ ]:
if running_on_colab:
    def draw_geometries(geometries):
        graph_objects = []

        for geometry in geometries:
            geometry_type = geometry.get_geometry_type()

            if geometry_type == o3d.geometry.Geometry.Type.PointCloud:
                points = np.asarray(geometry.points)
                colors = None
                if geometry.has_colors():
                    colors = np.asarray(geometry.colors)
                elif geometry.has_normals():
                    colors = (0.5, 0.5, 0.5) + np.asarray(geometry.normals) * 0.5
                else:
                    geometry.paint_uniform_color((1.0, 0.0, 0.0))
                    colors = np.asarray(geometry.colors)

                scatter_3d = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=1, color=colors))
                graph_objects.append(scatter_3d)

            if geometry_type == o3d.geometry.Geometry.Type.TriangleMesh:
                triangles = np.asarray(geometry.triangles)
                vertices = np.asarray(geometry.vertices)
                colors = None
                if geometry.has_triangle_normals():
                    colors = (0.5, 0.5, 0.5) + np.asarray(geometry.triangle_normals) * 0.5
                    colors = tuple(map(tuple, colors))
                else:
                    colors = (1.0, 0.0, 0.0)

                mesh_3d = go.Mesh3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], i=triangles[:,0], j=triangles[:,1], k=triangles[:,2], facecolor=colors, opacity=0.50)
                graph_objects.append(mesh_3d)

        fig = go.Figure(
            data=graph_objects,
            layout=dict(
                scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'
                )
            )
        )
        fig.show()

In [ ]:
if running_on_colab:
    o3d.visualization.draw_geometries = draw_geometries

anchor_color   = [0, 0, 1.0] # blue
positive_color = [0, 1.0, 0] # green
negative_color = [1.0, 0, 0] # red
p_colors = [anchor_color, positive_color, negative_color]

def visualize( pointcloud:np.array =None,
               anchor    :np.array =None,
               positive  :np.array =None,
               negative  :np.array =None,
               radius    :np.array =None,
               rot_mat   :np.array =None):
    """
    INPUT
      pointcloud : numpy array of 3D points
      anchor     : anchor point
      positive   :
      negative   :
    """
    geoms = []
    if pointcloud is not None:
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(pointcloud)
        pcd.paint_uniform_color([0.6, 0.6, 0.6])
        if rot_mat is not None:
            pcd.rotate(rot_mat, center=(0, 0, 0))
        geoms.append(pcd)
    for point, color in zip([anchor, positive, negative], p_colors):
        if point is not None:
            assert radius is not None
            sphere = o3d.geometry.TriangleMesh.create_sphere(radius=radius, resolution=20)
            sphere.translate(point)
            sphere.paint_uniform_color(color)
            geoms.append(sphere)
    # visualize the colored point cloud
    o3d.visualization.draw_geometries(geoms)

# PointCloud Dataset

`torch.utils.data.Dataset` is an abstract class representing a dataset. Your custom dataset should inherit `Dataset` and override the following methods:

* `__init__` to initialize your dataset. For example, if your dataset fits in memory, you can load the entire dataset in a list, or you can just store the list of dataset files.
* `__len__` so that len(dataset) returns the size of the dataset.
* `__getitem__` to support indexing such that `dataset[i]` can be used to get  the i-th sample

Therefore, the structure of the class is:

```
class CustomDataset(Dataset):

    def __init__(self, init_parameters):
        self.param1 = param1
        [...]

    def __len__(self):
        [...]

    def __getitem__(self, idx):
        [...]

        return sample[idx]
```


In [ ]:
class PointCloudData(Dataset):
    def __init__(self,
                dataset_path: str,
                samples_per_epoch: int,
                points_to_sample:int = 200000,
                radius:float =0.02,
                min_dist=1.5e-2,
                N = 750,
                noise_mean=0,
                noise_variance = 6e-5,
                is_test_set=False):
        """
          INPUT
              dataset_path: path to the dataset folder
              transform   : transform function to apply to point cloud
        """

        self.radius = radius
        self.min_dist = min_dist
        self.N = N
        self.samples_per_epoch = samples_per_epoch
        self.points_to_sample = points_to_sample
        self.noise_mean = noise_mean
        self.noise_variance = noise_variance
        self.is_test_set = is_test_set

        # _n means noised version
        self.mesh = []
        self.pcds, self.pcds_n = [], []
        self.KDtrees, self.KDtrees_n = [], []

        ## if it's the test set, pre-define a random rotation matrix
        if self.is_test_set:
            self.common_rot_mat = self.get_xyz_random_rotation()

        for file in glob.glob(dataset_path + "/*.ply"):
            print("parsing file", file)
            mesh = o3d.io.read_triangle_mesh(file)
            pcd = mesh.sample_points_uniformly(self.points_to_sample)
            if self.is_test_set:
                pcd.rotate(self.common_rot_mat.as_matrix(), center=(0, 0, 0))
            pcd_tree = o3d.geometry.KDTreeFlann(pcd)

            pcd_n = self.apply_noise(mesh.sample_points_uniformly(self.points_to_sample), self.noise_mean, self.noise_variance)
            if self.is_test_set:
                pcd_n.rotate(self.common_rot_mat.as_matrix(), center=(0, 0, 0))
            pcd_n_tree = o3d.geometry.KDTreeFlann(pcd_n)

            self.mesh.append(mesh)

            self.pcds.append(np.asarray(pcd.points))
            self.pcds_n.append(np.asarray(pcd_n.points))

            self.KDtrees.append(pcd_tree)
            self.KDtrees_n.append(pcd_n_tree)

    # function to apply noise
    def apply_noise(self, pcd, mu, sigma):
        noisy_pcd = copy.deepcopy(pcd)
        points = np.asarray(noisy_pcd.points)
        points += np.random.normal(mu, sigma, size=points.shape)
        noisy_pcd.points = o3d.utility.Vector3dVector(points)
        return noisy_pcd

    def __len__(self):
        return self.samples_per_epoch

    def get_xyz_random_rotation(self):
        random_rotation_on_xyz_axis = np.random.rand(3) * 2 * np.pi
        return R.from_euler('xyz', random_rotation_on_xyz_axis, degrees=False)

    def get_point_cloud_center(self, pc_points):
        return pc_points.mean(axis=0)

    def apply_rotation(self, point, rot_mat, pcd_center):
        return np.dot(point.reshape(1, 3), rot_mat.as_matrix().T)[0, :]

    def apply_rotation_pc(slef, points, rot_mat, pcd_center):
        return np.dot(points, rot_mat.as_matrix().T)

    def bufferize_pointcloud(self, points, N):
        pc = np.zeros((self.N, 3), dtype=np.float32)
        pc[:min(self.N, points.shape[0]), :] = points[:min(self.N, points.shape[0]), :]
        return pc

    def sample_anchor_point(self, pcd_points, pcd_tree):
        ############# START ###########
        ######### COMPLETE HERE #######
        anchor_pt = random.choice(pcd_points)
        print(anchor_pt)
        anchor_neighborhood_idxs = pcd_tree.search_radius_vector_3d(anchor_pt, self.radius)[1]

        ############# END #############
        return anchor_pt, anchor_neighborhood_idxs

    def sample_positive_point(self, pcd_n_points, pcd_n_tree, anchor_pt):
        _, noisy_anchor_nn_idx, _ = pcd_n_tree.search_knn_vector_3d(anchor_pt, 1)

        ############# START ###########
        ######### COMPLETE HERE #######
        pos_pt = pcd_n_points[noisy_anchor_nn_idx[0]]
        noisy_positive_neighborhood_idxs = pcd_n_tree.search_radius_vector_3d(pos_pt, self.radius)[1]
        ############# END #############

        return pos_pt, noisy_positive_neighborhood_idxs

    def sample_negative_point(self, pcd_n_points, pcd_n_tree, anchor_pt):
        ############# START ###########
        ######### COMPLETE HERE #######
        ##########TO CHECK!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        flag = True

        while flag:  #if neg_pt not found, pick a random point and check if its distance from anchor_pt is greater than min_dist
          random_idx = np.random.randint(0, len(pcd_n_points))
          random_point_guess = pcd_n_points[random_idx]
          squared_dist = np.sum((random_point_guess - anchor_pt)**2, axis = 0)
          dist = np.sqrt(squared_dist)

          if dist > self.min_dist:
            #negative point found
            flag = False
            neg_pt = random_point_guess

        _,noisy_negative_neighborhood_idxs,_ = pcd_n_tree.search_radius_vector_3d(neg_pt, self.radius) #finding the neighbors
        return neg_pt, noisy_negative_neighborhood_idxs

        ############# END #############

    def get_sampled_pointcloud(self, mesh_idx, N):
        idxs = np.arange(0, self.pcds[mesh_idx].shape[0], 1)
        np.random.shuffle(idxs)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(self.pcds[mesh_idx][idxs[:N]])
        return pcd

    def generate_test_set(self, mesh_idx, N=500):

        self.test_points = self.pcds[mesh_idx]

        ## sample test points randomly
        idxs = np.arange(0, self.test_points.shape[0], 1)
        np.random.shuffle(idxs)
        self.test_points_sampled = self.test_points[idxs[:N]]

        # ## as a bonus study: you can sample keypoints from the point cloud
        # ## using open3d and analyze the results. Keypoints are inherently easier.
        # pcd = o3d.geometry.PointCloud()
        # pcd.points = o3d.utility.Vector3dVector(self.pcds[mesh_idx])
        # keypoints = o3d.geometry.keypoint.compute_iss_keypoints(pcd,
        #                                                 salient_radius=0.005,
        #                                                 non_max_radius=0.005,
        #                                                 gamma_21=0.4,
        #                                                 gamma_32=0.5)
        # self.test_points_sampled = np.random.permutation(np.asarray(keypoints.points))

        self.test_tree = self.KDtrees[mesh_idx]

    def generate_noisy_test_set(self, mesh_idx):
        self.test_points_n = self.pcds_n[mesh_idx]
        self.test_tree_n = self.KDtrees_n[mesh_idx]

        test_points_sampled_n = []
        for i in tqdm(range(self.test_points_sampled.shape[0])):
            _, gt_nn_idx, _ = self.test_tree_n.search_knn_vector_3d(self.test_points_sampled[i], 1)
            gt_nearest_point = self.test_points_n[gt_nn_idx].squeeze()
            test_points_sampled_n.append(gt_nearest_point)

        self.test_points_sampled_n = np.asarray(test_points_sampled_n)

    def compute_descriptors(self, tinypointnet, noisy=False):
        tinypointnet.eval()

        if noisy:
            queries = self.test_points_sampled_n
            tree    = self.test_tree_n
            points  = self.test_points_n
        else:
            queries = self.test_points_sampled
            tree    = self.test_tree
            points  = self.test_points
        tot_queries = queries.shape[0]

        descriptors = np.zeros((tot_queries, 256), dtype=np.float32)
        for i in tqdm(range(tot_queries)):
            pt = queries[i]

            ## find neighborhood of points
            _, idx, _ = tree.search_radius_vector_3d(pt, self.radius)
            point_set = points[idx]

            ## normalize the points
            point_set = (point_set - pt)

            pc = np.zeros((self.N, 3), dtype=np.float32)
            pc[:min(self.N, point_set.shape[0]), :] = point_set[:min(self.N, point_set.shape[0]), :]

            # transform
            anchor  = torch.from_numpy(pc).unsqueeze(0).float().transpose(1,2).to(device)
            descriptors[i, :] = tinypointnet(anchor)[0, :, 0].cpu().detach().numpy()
        return descriptors

    def __getitem__(self, _):
        mesh_idx = np.random.randint(0, len(self.mesh))

        pcd_points = self.pcds[mesh_idx]        ## anchor will be drawn from this
        pcd_n_points = self.pcds_n[mesh_idx]    ## positive and negative will be drawn from this

        # ANCHOR: select a random anchor point
        anchor_pt, anchor_neighborhood_idxs = self.sample_anchor_point(pcd_points, self.KDtrees[mesh_idx])

        # POSITIVE: find corresponding point in the noisy point cloud
        pos_pt, noisy_positive_neighborhood_idxs = self.sample_positive_point(pcd_n_points, self.KDtrees_n[mesh_idx], anchor_pt)

        # NEGATIVE: find far point (at least at distance min_dist)
        neg_pt, noisy_negative_neighborhood_idxs = self.sample_negative_point(pcd_n_points, self.KDtrees_n[mesh_idx], anchor_pt)
        if neg_pt is None: ## it should never fail, but if it fails: restart experiment
            quit("FAIL: restart experiment")


        # get points
        point_set_anchor   = pcd_points[anchor_neighborhood_idxs]
        point_set_positive = pcd_n_points[noisy_positive_neighborhood_idxs]
        point_set_negative = pcd_n_points[noisy_negative_neighborhood_idxs]

        if not self.is_test_set:
            # generate a random rotation
            rot_mat = self.get_xyz_random_rotation()

            # apply the random rotation to point cloud and points
            pcd_points_center = self.get_point_cloud_center(pcd_points)
            pcd_n_points_center = self.get_point_cloud_center(pcd_n_points)
            anchor_pt = self.apply_rotation(anchor_pt, rot_mat, pcd_points_center)
            pos_pt    = self.apply_rotation(pos_pt, rot_mat, pcd_n_points_center)
            neg_pt    = self.apply_rotation(neg_pt, rot_mat, pcd_n_points_center)
            point_set_anchor   = self.apply_rotation_pc(point_set_anchor, rot_mat, pcd_points_center)
            point_set_positive = self.apply_rotation_pc(point_set_positive, rot_mat, pcd_n_points_center)
            point_set_negative = self.apply_rotation_pc(point_set_negative, rot_mat, pcd_n_points_center)
        else:
            rot_mat = self.common_rot_mat

        # center points around their centroid
        point_set_anchor   = (point_set_anchor - anchor_pt)
        point_set_positive = (point_set_positive - pos_pt)
        point_set_negative = (point_set_negative - neg_pt)

        # copy points coordinates to a fixed dimension np.array
        point_set_anchor   = self.bufferize_pointcloud(point_set_anchor  , self.N)
        point_set_positive = self.bufferize_pointcloud(point_set_positive, self.N)
        point_set_negative = self.bufferize_pointcloud(point_set_negative, self.N)

        # transform from numpy to torch.Tensor
        point_set_anchor    = torch.from_numpy(point_set_anchor)
        point_set_positive  = torch.from_numpy(point_set_positive)
        point_set_negative  = torch.from_numpy(point_set_negative)

        return mesh_idx, point_set_anchor, point_set_positive, point_set_negative, anchor_pt, pos_pt, neg_pt, rot_mat.as_matrix()


# Dataset Creation

Now we can instantiate our training and test dataset objects.

In [ ]:
train_ds  = PointCloudData(dataset_path_train, samples_per_epoch=500)
valid_ds  = PointCloudData(dataset_path_valid, samples_per_epoch=500)
test_ds   = PointCloudData(dataset_path_test,  samples_per_epoch=500, is_test_set=True)

parsing file /content/drive/MyDrive/3D_dp_dataset/3dshapes/train/bun_zipper.ply
parsing file /content/drive/MyDrive/3D_dp_dataset/3dshapes/train/dragon_vrip.ply
parsing file /content/drive/MyDrive/3D_dp_dataset/3dshapes/train/Armadillo.ply
parsing file /content/drive/MyDrive/3D_dp_dataset/3dshapes/valid/bun_zipper.ply
parsing file /content/drive/MyDrive/3D_dp_dataset/3dshapes/valid/dragon_vrip.ply
parsing file /content/drive/MyDrive/3D_dp_dataset/3dshapes/valid/Armadillo.ply
parsing file /content/drive/MyDrive/3D_dp_dataset/3dshapes/test/happy_vrip.ply


Creating a `Dataset` class may seem unnecessary for the most basic problems. But it really helps when the dataset and the training procedure start to get more complex.

One of the most useful benefit of defining a `Dataset` class is the possiblity to use the PyTorch `Dataloader` module.

By operating on the dataset directly, we are losing out on a lot of features by using a simple for loop to iterate over the data. In particular, we are missing out on:

* Batching the data
* Shuffling the data
* Load the data in parallel using multiprocessing workers.

`torch.utils.data.DataLoader` is an iterator which provides all these features. Parameters used below should be clear.

In [ ]:
# warning: batch_size needs to be at least 2
train_loader  = DataLoader( dataset=train_ds,  batch_size=50, shuffle=True  )
valid_loader  = DataLoader( dataset=valid_ds,  batch_size=50, shuffle=False )
test_loader   = DataLoader( dataset=test_ds,   batch_size=20, shuffle=False )

# Visualize some data

In [ ]:
ds = train_ds

for (mesh_idx, _, _, _, anchor, positive, negative, rot_mat) in ds:
    idx = mesh_idx
    visualize(ds.pcds[idx],
              anchor = anchor,
              positive = positive,
              negative = negative,
              radius=ds.radius,
              rot_mat=rot_mat)
    break


Output hidden; open in https://colab.research.google.com to view.

# Network Definition

## Network Base Module

A network is defined by extending the *torch.nn.module* class. The basic structure is:

```
class Net(nn.Module):
    
    def __init__(self, input_parameters):
        super().__init__() # This executes the parent __init__ method
        [...]

    def forward(self, x, optional_parameters):
        [...]
        return out # return the output of the network
```

You need to define two methods:
*   **\_\_init\_\_**: The constructor method. This is exectuted when the object is initialized (no need to call it explicitly). Here you have to instantiate all the network's parameters. PyTorch provides utility functions to easily initialize most of the commonly used deep learning layers.
*   **forward**: Here you define the forward pass of the network, from the input *x* to the output (the method must return the network output). You just need to define the forward part, the back-propagation is automatically tracked by the framework!

In [ ]:
# Multi Layer Perceptron
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.input_size   = input_size
        self.output_size  = output_size
        self.conv  = nn.Conv1d(self.input_size, self.output_size, 1)
        self.bn    = nn.BatchNorm1d(self.output_size)

    def forward(self, input):
        return F.relu(self.bn(self.conv(input)))

# Fully Connected with Batch Normalization
class FC_BN(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.input_size   = input_size
        self.output_size  = output_size
        self.lin  = nn.Linear(self.input_size, self.output_size)
        self.bn    = nn.BatchNorm1d(self.output_size)

    def forward(self, input):
        return F.relu(self.bn(self.lin(input)))

class TNet(nn.Module):
    def __init__(self, k=3):
        super().__init__()
        self.k=k

        self.mlp1 = MLP(self.k, 64)
        self.mlp2 = MLP(64, 128)
        self.mlp3 = MLP(128, 1024)

        self.fc_bn1 = FC_BN(1024, 512)
        self.fc_bn2 = FC_BN(512,256)

        self.fc3 = nn.Linear(256,k*k)


    def forward(self, input):
        # input.shape == (batch_size,n,3)

        bs = input.size(0)
        xb = self.mlp1(input)
        xb = self.mlp2(xb)
        xb = self.mlp3(xb)

        pool = nn.MaxPool1d(xb.size(-1))(xb)
        flat = nn.Flatten(1)(pool)

        xb = self.fc_bn1(flat)
        xb = self.fc_bn2(xb)

        #initialize as identity
        init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
        if xb.is_cuda:
            init=init.cuda()
        matrix = self.fc3(xb).view(-1,self.k,self.k) + init
        return matrix

In [ ]:
class TinyPointNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_transform   = TNet(k=3) ## if you use the shot rotation matrix, this is not going to be used
        self.feature_transform = TNet(k=64)

        ############# START ###########
        ######### COMPLETE HERE #######
        self.mlp1 = MLP(3, 64)
        self.mlp2 = MLP(64, 64)
        #here is put the feature transform TNet(64)
        self.mlp3 = MLP(64, 64)
        self.mlp4 = MLP(64, 128)
        self.mlp5 = MLP(128, 256)
        ############# END #############

    def forward(self, input):
        n_pts = input.size()[2]
        # matrix3x3 = self.input_transform(input)
        matrix3x3 = self.shot_canonical_rotation(input, 3)
        input_transform_output = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)

        ############# START ###########
        ######### COMPLETE HERE #######

        #first mlp
        x = self.mlp1(input_transform_output)
        x = self.mlp2(x)

        #feature transform
        matrix64x64 = self.feature_transform(x)
        feature_transform_output = torch.bmm(torch.transpose(x,1,2), matrix64x64).transpose(1,2)

        #second mlp
        x2 = self.mlp3(feature_transform_output)
        x2 = self.mlp4(x2)
        x2 = self.mlp5(x2)

        #global feature
        pool = nn.MaxPool1d(x2.size(-1))(x2)
        global_feature = nn.Flatten(1)(pool)

        ############# END #############

        return global_feature

    def shot_canonical_rotation(self, input, k):
        # input.shape == (batch_size, n, k)
        batch_size, n, _ = input.size()
        rotation_matrices = []

        for i in range(batch_size):
            features_batch = input[i]
            centroid = torch.mean(features_batch, dim=1, keepdim=True)
            centered_features = features_batch - centroid

            if torch.any(torch.isnan(centered_features)) or torch.any(torch.isinf(centered_features)):
                centered_features[torch.isnan(centered_features) | torch.isinf(centered_features)] = 1e-6

            distances = torch.norm(centered_features, dim=0)

            weights = 1.0 - distances
            weights[weights < 0] = 0

            weights_sum = torch.sum(weights)
            if weights_sum > 0:
                weights = weights / weights_sum
            else:
                weights = torch.ones_like(weights) / n

            weighted_cov_matrix = torch.zeros((centered_features.size(0), centered_features.size(0)), device=input.device)

            ############# START ###########
            ######### COMPLETE HERE #######
            ### compute the covariance matrix

            for j in range(centered_features.size(1)):
              #multiplying the j-th vector with itself and weight it.
              #sum the result at each iteration
              weighted_cov_matrix += weights[j] * torch.ger(centered_features[:, j], centered_features[:, j])

            ############# END #############

            weighted_cov_matrix += torch.eye(weighted_cov_matrix.size(-1), device=input.device) * 1e-6

            ############# START ###########
            ######### COMPLETE HERE #######
            ### compute the eigenvectors of covariance matrix
            eigvals, eigvecs = torch.linalg.eig(weighted_cov_matrix)
            V = torch.real(eigvecs)
            ############# END #############

            rotation_matrix = V[:, :k]

            for j in range(k):
                signs = torch.sign(torch.sum(centered_features * rotation_matrix[:, j].unsqueeze(1), dim=0))
                if torch.sum(signs >= 0) < torch.sum(signs < 0):
                    rotation_matrix[:, j] = -rotation_matrix[:, j]

            rotation_matrices.append(rotation_matrix)

        return torch.stack(rotation_matrices, dim=0)


# Training loop

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
tinypointnet = TinyPointNet()
tinypointnet.to(device);

In [ ]:
optimizer = torch.optim.Adam(tinypointnet.parameters(), lr=0.0005)

In [ ]:
def train(model, train_loader, valid_loader=None,  epochs=45, save=True):
    best_valid_loss = 1e10
    ############## START #############
    ####### COMPLETE THIS PART: put the correct loss function #######
    # tinypointnetloss =
    tinypointnetloss = nn.TripletMarginLoss(margin = 1, p = 2)
    ############## END ###############

    # these lists keep track of the losses across epochs
    train_losses, valid_losses = [], []

    for epoch in range(epochs):
        # local list of losses
        train_loss, valid_loss = [], []

        # train
        tinypointnet.train()

        for (_, anchor, positive, negative, _, _, _, _) in tqdm(train_loader):

            # retrieve anchors, positives and negatives batch
            anchor   =   anchor.to(device).float().transpose(1,2)
            positive = positive.to(device).float().transpose(1,2)
            negative = negative.to(device).float().transpose(1,2)

            optimizer.zero_grad()

            # let PointNetTiny model compute the descriptors
            anchor_desc   = tinypointnet(anchor)
            positive_desc = tinypointnet(positive)
            negative_desc = tinypointnet(negative)

            # compute the loss associated to these descriptors
            loss = tinypointnetloss(anchor_desc, positive_desc, negative_desc)

            # Backpropagate the gradient
            loss.backward()
            optimizer.step()

            # Keep track of the statistics
            train_loss.append(loss.item())
            #pbar.set_postfix(loss=curr_loss)

        train_loss = np.asarray(train_loss).mean()
        print(f'epoch {epoch} - train loss:', train_loss)
        train_losses.append(train_loss)

        # validation
        tinypointnet.eval()
        pbar = tqdm(valid_loader, leave=False)

        with torch.no_grad():
            for (_, anchor, positive, negative, _, _, _, _) in pbar:
                pbar.set_description(f"valid - epoch {epoch}")

                anchor   =   anchor.to(device).float().transpose(1,2)
                positive = positive.to(device).float().transpose(1,2)
                negative = negative.to(device).float().transpose(1,2)

                anchor_desc   = tinypointnet(anchor)
                positive_desc = tinypointnet(positive)
                negative_desc = tinypointnet(negative)
                loss = tinypointnetloss(anchor_desc, positive_desc, negative_desc)
                curr_loss = loss.item()

                valid_loss.append(curr_loss)

                pbar.set_postfix(loss=curr_loss)

        valid_loss = np.asarray(valid_loss).mean()
        print(f'epoch {epoch} - valid loss:', valid_loss)
        valid_losses.append(valid_loss)

        # save the model
        if save and valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            if running_on_colab:
                path = os.path.join(drive_path, "MyDrive", "tinypointnetmodel.yml")
            else:
                path = os.path.join("tinypointnetmodel.yml")
            print("best_valid_loss:", best_valid_loss, "saving model at", path)
            torch.save(tinypointnet.state_dict(), path)
    return train_losses, valid_losses


In [ ]:
train_losses, valid_losses = train(tinypointnet, train_loader, valid_loader, save=True)

  0%|          | 0/10 [00:00<?, ?it/s]

[0.02118356 0.06081333 0.00066362]
[-0.06226274  0.16234165 -0.00546839]
[-0.11752626  0.12439678 -0.08867788]
[ 0.04154347  0.07838293 -0.03451008]
[ 0.06093543  0.11911797 -0.02465138]
[ 0.01165225  0.12174555 -0.01178193]
[0.05367696 0.07016441 0.02365602]
[-0.03256946  0.17129648 -0.00341518]
[-0.08093063  0.14948341  0.00112495]
[-0.05853326  0.03341388  0.0002299 ]
[-0.08236307  0.16740031  0.01461104]
[-0.05315863  0.12259656  0.00457263]
[-0.0379737   0.05686597  0.0107173 ]
[-0.0220818   0.16453667 -0.04876422]
[ 0.05846786  0.10960531 -0.02450898]
[0.03233044 0.11589014 0.0246607 ]
[0.0723968  0.06465304 0.02104183]
[-0.09499795  0.15146925 -0.00063376]
[-0.05320256  0.14587757  0.02242842]
[-0.06647042  0.12003394 -0.01981765]
[-0.02027099  0.08805853 -0.0327744 ]
[-0.05752119  0.12134773  0.00541567]
[-0.03308624  0.17362996 -0.02216499]
[-0.0434665   0.1577074   0.00543053]
[0.05484195 0.07966872 0.00989012]
[-0.05876302  0.04362043  0.04398399]
[-0.04467478  0.03357154  0

 10%|█         | 1/10 [00:14<02:08, 14.25s/it]

[ 0.0966249   0.13601073 -0.04682409]
[-0.06278749  0.12310824 -0.00881266]
[-0.05652971  0.12002832 -0.01079849]
[-0.09296332  0.12454482  0.01058453]
[-0.05941052  0.03462263  0.0430127 ]
[-0.08179     0.07381881  0.01110842]
[0.00745475 0.06305899 0.0225866 ]
[0.08121012 0.08565134 0.00110658]
[-0.06889208  0.06774746  0.03038403]
[ 0.04011349 -0.04212461  0.0078768 ]
[-0.07522951 -0.03187969  0.03395692]
[-0.0360034   0.1022727  -0.02159431]
[0.04527444 0.07558214 0.02203384]
[-0.03838656  0.18794849 -0.01626665]
[-0.08656142  0.10779482  0.01559935]
[0.07212757 0.08867763 0.0080738 ]
[-0.10328236  0.09855238 -0.03843243]
[-0.04879718  0.15834428 -0.00049826]
[ 0.06080884 -0.03481544  0.0288991 ]
[0.02920657 0.04595316 0.03455138]
[ 0.02271824  0.06600669 -0.02988096]
[-0.06719613  0.06214872 -0.02655371]
[-0.03456153  0.05289276 -0.03608317]
[-0.06614077  0.17890211 -0.01329243]
[-0.01248144  0.01495865  0.00523563]
[0.03173236 0.06516057 0.04067143]
[ 0.03625404  0.0905761  -0.03

 20%|██        | 2/10 [00:26<01:45, 13.15s/it]

...

[-0.05622584  0.10701561  0.03029413]
[ 0.07588644  0.07101868 -0.00010885]
[-0.03690237  0.1426442   0.00174895]
[-0.02418696  0.12245349  0.02624566]
[ 0.02976827  0.15730548 -0.01668588]
[-0.0325117   0.09487794 -0.00410143]
[-0.03860569  0.08195218 -0.02816387]
[-0.09178135  0.12288494  0.00746238]
[ 0.04430187 -0.09392433  0.05475383]
[ 0.00100698 -0.01003014  0.0769826 ]
[ 0.05098498 -0.03313187  0.04192248]
[-0.00264545  0.09024888  0.00119567]
[-0.01764282  0.17469598 -0.04372397]
[0.05080803 0.05823414 0.03011479]
[-0.02429864  0.15042204 -0.02574528]
[ 0.02066571  0.05992124 -0.00115166]
[-0.11087435  0.11970977 -0.02238588]
[0.00874642 0.12370358 0.03384587]
[-0.02144746  0.07358146 -0.038765  ]
[-0.01657509  0.18410973 -0.0292073 ]
[-0.08148706 -0.10274498  0.02925354]
[ 0.02979816  0.0540125  -0.02165112]
[ 0.01849996  0.0457791  -0.02261864]
[-0.07794513  0.13036923 -0.00629183]
[0.0474768  0.04514194 0.00090909]
[-0.06266779  0.03460401  0.03969547]
[ 0.05443711  0.06330

 30%|███       | 3/10 [00:37<01:26, 12.35s/it]

[ 0.00488675  0.05688485 -0.03085763]
[0.07190224 0.10328672 0.01025147]
[ 0.03310344  0.07157629 -0.01755364]
[ 0.02395076 -0.09791888  0.04767149]
[-0.02136042  0.1625709  -0.05385701]
[-0.07813242  0.09988368 -0.03015356]
[-0.02589369 -0.00121979  0.03797134]
[-0.04008717  0.06755812  0.02560749]
[-0.08955035  0.09929885 -0.04830283]
[-0.04517376 -0.02148205  0.02813974]
[0.02660438 0.05488523 0.05170248]
[-0.05406576  0.09931126 -0.02046525]
[ 0.09669009  0.09714139 -0.01712966]
[0.00047372 0.16788071 0.04267163]
[-0.01377247  0.14922052 -0.05276453]
[-0.02816816  0.12506456  0.01733272]
[0.01398982 0.05798031 0.01819038]
[-0.04061629  0.08559104 -0.01993174]
[-0.08423319  0.0638848  -0.03567241]
[-0.06249995  0.05336843 -0.00788483]
[-0.01503065  0.07806604  0.06473817]
[-0.05561928  0.07608114  0.03068152]
[-0.01185716  0.08651393  0.05707335]
[-0.07420806  0.06611002  0.00566553]
[-0.09458498  0.16671245 -0.00685641]
[-0.04785874  0.03461592  0.04072104]
[-0.08910775  0.15325505

 40%|████      | 4/10 [00:48<01:11, 11.97s/it]

[0.036352   0.13816403 0.05285376]
[-0.04128694  0.08446256  0.0427279 ]
[ 0.0550017   0.05380008 -0.00690751]
[-0.04313116  0.10933567 -0.01882729]
[0.01418251 0.10870726 0.04067375]
[-0.01786853  0.15672602 -0.02079874]
[ 0.02940277  0.15427029 -0.02968523]
[-0.06630051  0.11093655  0.03801276]
[-0.04847186  0.0943214   0.01211106]
[-0.04299664  0.12850652  0.01515383]
[-0.05908101  0.05605263  0.01017119]
[ 0.11106182  0.14696614 -0.07817261]
[-0.08388641  0.14239599  0.04218933]
[ 0.09922648  0.16789459 -0.05041238]
[0.01970817 0.06580202 0.00914454]
[ 0.00667588  0.10869162 -0.02010982]
[-0.02798124  0.09789505 -0.02348026]
[0.05940458 0.07862812 0.00217915]
[-0.06222506  0.06395436 -0.00581135]
[-0.00659762  0.13031245 -0.00338865]
[ 0.01801031  0.13088254 -0.05078084]
[-0.09630181  0.11988363 -0.06543426]
[-0.07534766 -0.06386231  0.01551026]
[-0.04498568  0.06734244 -0.01553643]
[-0.0269882   0.15236212 -0.01391766]
[-0.08808998  0.13923723 -0.04933104]
[ 0.00475244  0.12527236

## Visualize the Training trend

In [ ]:
def numpy_ewma_vectorized_v2(data, window=10):
    # Return the Exponentially Weighted Moving Average
    # for better visualizing the "trend" of the metrics

    alpha = 2 /(window + 1.0)
    alpha_rev = 1-alpha
    n = data.shape[0]

    pows = alpha_rev**(np.arange(n+1))

    scale_arr = 1/pows[:-1]
    offset = data[0]*pows[1:]
    pw0 = alpha*alpha_rev**(n-1)

    mult = data*pw0*scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums*scale_arr[::-1]
    return out

### Plot the trend of train and valid losses
By using an Exponentially Weighted Moving Average of the losses,
we can better understand the behaviour of our network by filtering out
the noisy *local values* and focusing more on the *global trend*.

In [ ]:
plt.plot(train_losses, label="train losses", color='r', alpha=0.2)
plt.plot(valid_losses, label="valid losses", color='b', alpha=0.2)
plt.plot(numpy_ewma_vectorized_v2(np.array(train_losses)), label="train losses EMA", color='r')
plt.plot(numpy_ewma_vectorized_v2(np.array(valid_losses)), label="valid losses EMA", color='b')
plt.legend()

# Test: feature matching

Let's compute the model test metrics

> First we need to load the best model weights





## Load checkpoint

In [ ]:
if running_on_colab:
    path = os.path.join(drive_path, "MyDrive", "tinypointnetmodel.yml")
else:
    path = os.path.join("tinypointnetmodel.yml")
tinypointnet = TinyPointNet()
tinypointnet.load_state_dict(torch.load(path))
tinypointnet.to(device)

## Prepare the test set

In [ ]:
test_ds.generate_test_set(0, 2000)

## build the ground truth nearest neighbors
## in other words, find thepoints in the noisy test points
## that are the nearest neighbors to the original, sampled, test points
test_ds.generate_noisy_test_set(0)


### Visualize the sampled test points
Little red spheres represent the test points sampled from the original test mesh

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(test_ds.test_points_sampled_n)
pcd.paint_uniform_color([0.5, 0.5, 0.5])
geoms = [pcd]

# visualize the colored point cloud
o3d.visualization.draw_geometries(geoms)

In [ ]:
pcd = test_ds.get_sampled_pointcloud(mesh_idx=0, N=5000)
pcd.paint_uniform_color([0.5, 0.5, 0.5])
geoms = [pcd]
for point in test_ds.test_points_sampled_n:
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1e-3)
    sphere.translate(point)
    sphere.paint_uniform_color([1, 0, 0])
    geoms.append(sphere)

# visualize the colored point cloud
o3d.visualization.draw_geometries(geoms)


### Compute the descriptor of each point in each point set

In [ ]:
descs   = test_ds.compute_descriptors(tinypointnet)
descs_n = test_ds.compute_descriptors(tinypointnet, noisy=True)

## matching time!
In other words, get the descriptor from the noisy point cloud which is closest to the query descriptor from the original point cloud

In [ ]:
pcd = test_ds.get_sampled_pointcloud(mesh_idx=0, N=5000)
pcd.paint_uniform_color([0.5, 0.5, 0.5])
geoms = [pcd]
for row in tqdm(range(test_ds.test_points_sampled_n.shape[0])):
    anchor       = test_ds.test_points_sampled[row]
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1e-3)
    sphere.translate(anchor)
    sphere.paint_uniform_color([1, 0, 0])
    geoms.append(sphere)


o3d.visualization.draw_geometries(geoms)


In [ ]:
correct = tot = 0

for row in tqdm(range(test_ds.test_points_sampled.shape[0])):
    desc = descs[row, :]
    dists = []
    anchor       = test_ds.test_points_sampled[row]
    true_near_pt = test_ds.test_points_sampled_n[row]

    for row2 in range(test_ds.test_points_sampled_n.shape[0]):
        desc2 = descs_n[row2, :]
        dist = np.linalg.norm(desc - desc2)
        dists.append(dist)

    min_row = np.argmin(np.asarray(dists))

    pred_pt = test_ds.test_points_sampled_n[min_row].squeeze()

    dist = np.linalg.norm(true_near_pt - pred_pt)

    # visualize(test_ds.test_points_sampled,
    #           anchor = anchor,
    #           positive = pred_pt,
    #           radius=test_ds.radius)

    if dist<test_ds.radius:
        correct += 1
    tot += 1

print()
print(f"accuracy: {correct*100/tot:6.3f}%")

